<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://gitlab.com/Manan-Sood" target="_blank"> Manan Sood </a> and <a href="https://www.cmpe.boun.edu.tr/~ozlem.salehi/" target="_blank"> Özlem Salehi </a> </td>
    </tr> 
 </table>
 
<hr>

# Simulated Annealing

Simulated annealing is a stochastic global search optimization algorithm.

The algorithm is inspired by annealing in _metallurgy_ where metal is heated to a high temperature quickly, then cooled slowly. 

The physical annealing process works by first exciting the atoms in the material at a high temperature, allowing the atoms to move around a lot, then decreasing their excitement slowly, allowing the atoms to fall into a new, more stable configuration. 

<b>Simulated annealing</b> (SA) mimics the physical annealing process. (We would like to point out that it is not a physical process but it is an analogy). 

It can be considered as a modified version of stochastic hill climbing. Stochastic hill climbing maintains a single candidate solution and takes steps of a random but constrained size from the candidate in the search space. If the new point is better than the current point, then the current point is replaced with the new point. This process continues for a fixed number of iterations.

<img src="../images/sa.gif" width="550">


Image is taken from Image is taken from [Wikipedia](https://en.wikipedia.org/wiki/Simulated_annealing) 

Simulated annealing executes the search in the same way. 

The main difference is that new points that are not as good as the current point (worse points) are accepted sometimes. A worse point is accepted probabilistically where the likelihood of accepting a solution worse than the current solution is a function of the _temperature_ of the search and how much worse the solution is than the current solution. Moving to worse solutions allow escaping from local minima. The temperature is decreased gradually, making unfavorable changes less probable as the process continues. 0 temperature is simply the hill climbing algorithm.  

## Algorithm

<B> Step 1: </B> Start with an initial feasible solution $s=s_0$ and temperature $t=t_0$.

<B> Step 2: </B> Until the termination conditions are reached, repeat the following: 
   
* Pick a solution $s'$ from the neighborhood of solutions $N(s)$. 
* Let $\bigtriangleup$ be the difference between cost of $s'$ and $s$.
* If $\bigtriangleup<0$, accept the new solution, i.e. $s=s'$. Otherwise, pick a random number $p$ between 0 and 1. Accept $s'$ if $e^{-\bigtriangleup c/t } >p$.
* Calculate the new temperature $t$ according to the <i>annealing schedule</i>.

### Some notes

- Annealing schedule describes how temperature decreases in time. Most common choices are linear ($t= t-a$), and geometric ($t = t*a)$.

- Neighborhood of solutions is obtained by altering the current state.

- Termination condition can be a fixed number of iterations or reaching some acceptable threshold of performance.

## D-Wave Simulated Annealer

Now we will investigate how we can run simulated annealing algorithm from D-Wave library neal.

In [1]:
from dimod import BQM

from neal import SimulatedAnnealingSampler 
sampler = SimulatedAnnealingSampler()

linear = {'x1': -5, 'x2': -3, 'x3': -8, 'x4': -6}
quadratic = {('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}
vartype = 'BINARY'

bqm = BQM(linear, quadratic, vartype)

sampleset = sampler.sample(bqm, num_reads=10)
print(sampleset)

  x1 x2 x3 x4 energy num_oc.
0  1  0  0  1  -11.0       1
1  1  0  0  1  -11.0       1
2  1  0  0  1  -11.0       1
3  1  0  0  1  -11.0       1
4  1  0  0  1  -11.0       1
5  1  0  0  1  -11.0       1
6  1  0  0  1  -11.0       1
7  1  0  0  1  -11.0       1
8  1  0  0  1  -11.0       1
9  1  0  0  1  -11.0       1
['BINARY', 10 rows, 10 samples, 4 variables]


In the example above, we use `SimulatedAnnealingSampler` to find the ground state of the `bqm`. 

One parameter we have used is the `num_reads`, which determines how many runs of the simulated annealing algorithm we would like to call. Each line in the output corresponds to solution found in one run of the algorithm.

There are also additional parameters you can provide such as `beta_schedule` and `num_sweeps` but we will not go into detail.  

Note that since the algorithm is stochastic, having multiple runs helps us to estimate better the minimum energy sample.

### Task 1

Find out what assignment of $x_1$ and $x_2$ minimizes the following objective function using simulated annealing. Set number of reads to 1000.

$$5x_1 + 7x_1 x_2 - 3x_2 + 2$$

In [3]:
# Your code here
linear = {'x1': 5, 'x2': -3}
quadratic = {('x1', 'x2'): 7}
offset = 2
vartype = 'BINARY'
bqm = BQM(linear, quadratic, offset, vartype)
sampleset = sampler.sample(bqm, num_reads=1000)
print(sampleset.first)

Sample(sample={'x1': 0, 'x2': 1}, energy=-1.0, num_occurrences=1)


[click here for solution](Simulated_Annealing_Solutions.ipynb#Task1)

There are additional parameters that you can define when running simualted annealing. You can check the whole list from [here](https://docs.ocean.dwavesys.com/projects/neal/en/latest/reference/generated/neal.sampler.SimulatedAnnealingSampler.sample.html#neal.sampler.SimulatedAnnealingSampler.sample).

It is also possible to input a QUBO dictionary for the sampler through the function `sample_qubo` and an Ising model by providing `h` and `J` using funcation `sample_ising`.

### Task 2

Use simulated annealing to find out the assignment that gives the minimum energy for the following QUBO dictionary. Set number of reads to 1000.

In [5]:
Q_dict = {
    ("x1", "x1"): 3,
    ("x2", "x2"): -7,
    ("x3", "x3"): 11,
    ("x4", "x4"): -1,
    ("x1", "x2"): 9,
    ("x1", "x3"): 1,
    ("x2", "x3"): 2,
    ("x3", "x4"): 8,
}

In [6]:
# Your code here
sampleset = sampler.sample_qubo(Q_dict, num_reads=1000)
print(sampleset.first)


Sample(sample={'x1': 0, 'x2': 1, 'x3': 0, 'x4': 1}, energy=-8.0, num_occurrences=1)


[click here for solution](Simulated_Annealing_Solutions.ipynb#Task2)

### Task 3

Use simulated annealing to find out the assignment that gives the minimum energy for the following Ising Model defined through `h` and `J` parameters. Set number of reads to 1000. 



In [8]:
h = {'s1': 3, 's2': 1,'s3': 4, 's4': 2}
J = {('s1', 's2'): 4, ('s1', 's3'): 1, ('s1', 's4'): 6, ('s3', 's4'): 7}

In [11]:
# Your code here
sampleset = sampler.sample_ising(h, J, num_reads=1000)
print(sampleset.first)

Sample(sample={'s1': -1, 's2': 1, 's3': -1, 's4': 1}, energy=-20.0, num_occurrences=1)


[click here for solution](Simulated_Annealing_Solutions.ipynb#Task3)